In [73]:
import pandas as pd
import numpy as np

In [35]:
datashare = pd.read_csv('data/datashare.csv')

In [37]:
align_data = pd.read_csv('datashare.csv')

In [50]:
datashare.loc[datashare['DATE'] >= 19570329]['DATE'].drop_duplicates()

0          19570329
1058       19570430
2117       19570531
3179       19570628
4247       19570731
             ...   
3731683    20160831
3737400    20160930
3743103    20161031
3748794    20161130
3754492    20161230
Name: DATE, Length: 718, dtype: int64

In [53]:
align_data.loc[(align_data['DATE'] <= 20161230) & (align_data['DATE'] >= 19570329)]['DATE'].shape[0]

3760033

In [59]:
for d in align_data.loc[(align_data['DATE'] <= 20161230) & (align_data['DATE'] >= 19570329)]['DATE'].drop_duplicates().to_list():
    if align_data.loc[align_data['DATE'] == d].shape[0] != datashare.loc[datashare['DATE'] == d].shape[0]:
        print(d)
        break

19600930


In [66]:
align_data.loc[align_data['DATE'] == 19600930].permno

47240    10006
47241    10014
47242    10030
47243    10057
47244    10065
         ...  
48350    53023
48351    58843
48352    68523
48353    75471
48354    86239
Name: permno, Length: 1115, dtype: int64

In [67]:
datashare.loc[datashare['DATE'] == 19600930].permno

45134    10006
45135    10014
45136    10030
45137    10057
45138    10065
         ...  
46243    53023
46244    58843
46245    68523
46246    75471
46247    86239
Name: permno, Length: 1114, dtype: int64

In [68]:
for sy in align_data.loc[align_data['DATE'] == 19600930].permno.to_list():
    if sy not in datashare.loc[datashare['DATE'] == 19600930].permno.to_list():
        print(sy)

25507


In [72]:
for d in 

,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,stdcf,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2
0,10006,19570329,8.630850e+04,1.117907,1.249717,0.134574,10.296745,0.024863,0.059540,0.002016,...,NaN,NaN,0.013856,1.557681e-07,0.029167,0.013546,0.914592,0.802461,1.066005e-07,37.0
1,10014,19570329,3.903375e+03,0.331304,0.109762,0.006667,7.032404,0.065248,0.059540,-0.086957,...,NaN,NaN,0.031389,8.383815e-06,0.071429,0.031165,0.943409,0.716406,1.105263e+00,NaN
2,10022,19570329,9.841000e+03,0.942052,0.887461,0.058761,7.294038,0.029338,0.059540,-0.037037,...,NaN,NaN,0.010066,5.360386e-06,0.020000,0.016886,0.993558,0.534808,2.387797e-07,NaN
3,10030,19570329,5.190100e+04,0.886532,0.785940,-0.032065,9.516942,0.022007,0.059540,0.000000,...,NaN,NaN,0.009523,1.090908e-07,0.017857,0.009104,0.736000,0.712647,9.202902e-08,NaN
4,10057,19570329,3.525000e+04,1.229520,1.511719,-0.102811,8.421013,0.025453,0.059540,-0.030717,...,NaN,NaN,0.007661,2.386914e-07,0.007194,0.006748,0.889503,0.775954,6.631579e+00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3760203,93428,20161230,1.250976e+06,1.595315,2.545028,-0.137348,14.916210,0.043201,0.016401,-0.001203,...,0.261325,6.0,0.029302,1.010214e-09,0.027604,0.017847,0.490189,5.347476,8.664791e-09,73.0
3760204,93429,20161230,5.600537e+06,0.298267,0.088963,0.206434,16.319190,0.026198,-0.071728,0.093973,...,0.169486,6.0,0.016001,1.093471e-10,0.040887,0.010437,0.539155,6.252920,8.275139e-09,62.0
3760205,93433,20161230,2.473625e+04,2.324893,5.405128,0.328248,9.975208,0.185537,-0.002451,0.279070,...,110.846951,2.0,0.149511,3.889972e-07,0.405473,0.137955,1.285707,29.078756,5.014198e-09,65.0
3760206,93434,20161230,8.573280e+04,0.628519,0.395036,0.020854,10.862196,0.059797,0.015187,-0.049505,...,0.860708,3.0,0.042877,8.576942e-08,0.031579,0.019838,0.629682,2.431419,3.518593e-08,1.0


In [71]:
mon_return.loc[(mon_return['date'] == 19600930) & (mon_return['permno'] == 25507)]

,permno,date,month,ret-rf


In [69]:
align_data.loc[(align_data['DATE'] == 19600930) & (align_data['permno'] == 25507)]

,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,stdcf,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2
48171,25507,19600930,NaN,0.741312,0.549543,0.604304,10.080906,0.05099,-0.039526,0.08547,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.0


1115

In [41]:
mon_return = pd.read_csv('data/month_return.csv')

In [49]:
mon_return['date'].drop_duplicates()

0          19570329
1058       19570430
2117       19570531
3177       19570628
4243       19570731
             ...   
3752114    20160831
3757797    20160930
3763452    20161031
3769115    20161130
3774780    20161230
Name: date, Length: 718, dtype: int64

In [ ]:
datashare.permno.drop_duplicates()

In [33]:
datashare.columns

Index(['permno', 'DATE', 'mvel1', 'beta', 'betasq', 'chmom', 'dolvol',
       'idiovol', 'indmom', 'mom1m', 'mom6m', 'mom12m', 'mom36m', 'pricedelay',
       'turn', 'absacc', 'acc', 'age', 'agr', 'bm', 'bm_ia', 'cashdebt',
       'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chinv',
       'chpmia', 'convind', 'currat', 'depr', 'divi', 'divo', 'dy', 'egr',
       'ep', 'gma', 'grcapx', 'grltnoa', 'herf', 'hire', 'invest', 'lev',
       'lgr', 'mve_ia', 'operprof', 'orgcap', 'pchcapx_ia', 'pchcurrat',
       'pchdepr', 'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt',
       'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc', 'ps',
       'quick', 'rd', 'rd_mve', 'rd_sale', 'realestate', 'roic', 'salecash',
       'saleinv', 'salerec', 'secured', 'securedind', 'sgr', 'sin', 'sp',
       'tang', 'tb', 'aeavol', 'cash', 'chtx', 'cinvest', 'ear', 'nincr',
       'roaq', 'roavol', 'roeq', 'rsup', 'stdacc', 'stdcf', 'ms', 'baspread',
       'ill', 'maxret', 'retvol', '

In [25]:
annual_chara = {
    'absacc': 1, 'acc': 2, 'age': 4, 'agr': 5, 'bm': 9,
    'bm_ia': 10, 'cashdebt': 12, 'cashpr': 13, 'cfp': 14, 'cfp_ia': 15,  
    'chatoia': 16, 'chcsho': 17, 'chempia': 18, 'chinv': 19, 'chpmia': 21,
    'convind': 24, 'currat': 25, 'depr': 26, 'divi': 27, 'divo': 28,
    'dy': 30, 'egr': 32, 'ep': 33, 'gma': 34, 'grcapx': 35,
    'grltnoa': 36, 'herf': 37, 'hire': 38, 'invest': 42, 'lev': 43,
    'lgr': 44, 'mve_ia': 52, 'operprof': 54, 'orgcap': 55, 'pchcapx_ia': 56,
    'pchcurrat': 57, 'pchdepr': 58, 'pchgm_pchsale': 59, 'pchquick': 60, 'pchsale_pchinvt': 61,
    'pchsale_pchrect': 62, 'pchsale_pchxsga': 63, 'pchsaleinv': 64, 'pctacc': 65, 'ps': 67, 
    'quick': 68, 'rd': 69, 'rd_mve': 70, 'rd_sale': 71, 'realestate': 72, 
    'roic': 77, 'salecash': 79, 'saleinv': 80, 'salerec': 81, 'secured': 82, 
    'securedind': 83, 'sgr': 84, 'sin': 85, 'sp': 86, 'tang': 91, 'tb': 92
}

In [23]:
annual_chara['pchurrat']

57

In [21]:
datashare.columns

Index(['permno', 'DATE', 'mvel1', 'beta', 'betasq', 'chmom', 'dolvol',
       'idiovol', 'indmom', 'mom1m', 'mom6m', 'mom12m', 'mom36m', 'pricedelay',
       'turn', 'absacc', 'acc', 'age', 'agr', 'bm', 'bm_ia', 'cashdebt',
       'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chinv',
       'chpmia', 'convind', 'currat', 'depr', 'divi', 'divo', 'dy', 'egr',
       'ep', 'gma', 'grcapx', 'grltnoa', 'herf', 'hire', 'invest', 'lev',
       'lgr', 'mve_ia', 'operprof', 'orgcap', 'pchcapx_ia', 'pchcurrat',
       'pchdepr', 'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt',
       'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc', 'ps',
       'quick', 'rd', 'rd_mve', 'rd_sale', 'realestate', 'roic', 'salecash',
       'saleinv', 'salerec', 'secured', 'securedind', 'sgr', 'sin', 'sp',
       'tang', 'tb', 'aeavol', 'cash', 'chtx', 'cinvest', 'ear', 'nincr',
       'roaq', 'roavol', 'roeq', 'rsup', 'stdacc', 'stdcf', 'ms', 'baspread',
       'ill', 'maxret', 'retvol', '

In [34]:
for chr in datashare.columns:
    if chr not in list(annual_chara.keys()) + list(quarter_chara.keys()) + list(month_chara.keys()):
        print(chr)

permno
DATE
sic2


In [10]:
quarter_chara = {
    'aeavol': 3,
    'cash': 11,
    'chtx': 22,
    'cinvest': 23,
    'ear': 31,
    'ms': 50,
    'nincr': 53,
    'roaq': 74,
    'roavol': 75,
    'roeq': 76,
    'rsup': 78,
    'stdacc': 89,
    'stdcf': 90
}

In [11]:
len(quarter_chara)

13

In [12]:
month_chara = {
    'baspread': 6,
    'beta': 7,
    'betasq': 8,
    'chmom': 20,
    'dolvol': 29,
    'idiovol': 39,
    'ill': 40,
    'indmom': 41,
    'maxret': 45,
    'mom12m': 46,
    'mom1m': 47,
    'mom36m': 48,
    'mom6m': 49,
    'mvel1': 51,
    'pricedelay': 66,
    'retvol': 73,
    'std_dolvol': 87,
    'std_turn': 88,
    'turn': 93,
    'zerotrade': 94
}

In [13]:
len(month_chara)

20